# Instruction Generation — Llama 3 Instruct 8B (lite fallback)
**TL;DR:** Demonstrate small instruction prompts on CPU with a TinyLlama fallback while documenting access needs for Llama 3.

**Models & Datasets:** [Meta-Llama-3-8B-Instruct (fallback: TinyLlama-1.1B-Chat)](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct) (Custom (Llama 3), Apache-2.0 (TinyLlama)), [Synthetic prompts (UltraChat sample)](https://huggingface.co/datasets/prompt-injection/ultra-chat) (CC BY-SA 4.0)
**Run Profiles:** 🖥️ CPU | 🍎 Metal (Apple Silicon) | 🧪 Colab/T4 | ⚡ CUDA GPU
**Env (minimal):** python>=3.10, transformers, datasets, evaluate, accelerate (optional: peft, bitsandbytes, timm, diffusers)
**Colab:** [Open in Colab](https://colab.research.google.com/github/SSusantAchary/Hands-On-Huggingface-AI-Models/blob/main/notebooks/nlp/instruct-generation-llama-3-instruct-8b_lite_cpu-first.ipynb)

**Switches (edit in one place):**
- `device` = {"cpu","mps","cuda"}
- `precision` = {"fp32","fp16","bf16","int8","4bit"}  (apply only if supported)
- `context_len` / `image_res` / `batch_size`

**Footprint & Speed (fill after run):**
- Peak RAM: TODO
- Peak VRAM: TODO (if GPU)
- TTFB: TODO, Throughput: TODO, Load time: TODO

**Gotchas:** bitsandbytes optional; CPU uses fp32 pipeline automatically ([Fixes & Tips](../fixes-and-tips/bitsandbytes-wheel-mismatch.md))



## Setup
Load the instruct model with a CPU-safe fallback if the full Llama 3 weights are inaccessible.


In [ ]:

import json
import os
import subprocess
import time
from pathlib import Path

import torch
from transformers import AutoTokenizer, pipeline

from notebooks._templates.measure import append_benchmark_row, measure_memory_speed

TARGET_MODEL = "meta-llama/Meta-Llama-3-8B-Instruct"
FALLBACK_MODEL = os.environ.get("HF_FALLBACK_MODEL", "TinyLlama/TinyLlama-1.1B-Chat-v1.0")
REQUESTED_MODEL = os.environ.get("HF_MODEL_ID", FALLBACK_MODEL)

DEVICE_PREFERENCE = os.environ.get("HF_DEVICE", "cpu")
PRECISION = os.environ.get("HF_PRECISION", "fp32")

def resolve_device(preference: str = "cpu") -> str:
    if preference == "cuda" and torch.cuda.is_available():
        return "cuda:0"
    if preference == "mps" and torch.backends.mps.is_available():
        return "mps"
    return "cpu"

DEVICE = resolve_device(DEVICE_PREFERENCE)
print(f"Using device={DEVICE}")

def choose_model() -> str:
    target = TARGET_MODEL if REQUESTED_MODEL == TARGET_MODEL else REQUESTED_MODEL
    if target == TARGET_MODEL:
        try:
            AutoTokenizer.from_pretrained(TARGET_MODEL, token=os.environ.get("HF_TOKEN"))
            print("Using Meta-Llama-3-8B-Instruct (ensure you accepted the license).")
            return TARGET_MODEL
        except Exception as error:  # noqa: BLE001
            print(f"Falling back to {FALLBACK_MODEL} because Llama 3 load failed: {error}")
    return FALLBACK_MODEL

MODEL_ID = choose_model()
OUTPUT_DIR = Path("outputs") / "instruct-generation"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

prompts = [
    "Summarise the repo's CPU-first philosophy in one sentence.",
    "Give me two bullet tips for speeding up Transformers inference on CPUs.",
    "When should I enable 4-bit quantization? Respond in 3 short bullet points.",
]


## Generate responses


In [ ]:

generation_kwargs = {
    "temperature": 0.7,
    "max_new_tokens": 128,
    "do_sample": True,
    "pad_token_id": None,
}

torch.manual_seed(42)
load_start = time.perf_counter()
generator = pipeline(
    "text-generation",
    model=MODEL_ID,
    device=DEVICE,
    torch_dtype=torch.float32,
)
load_time = time.perf_counter() - load_start

outputs = []
for prompt in prompts:
    response = generator(prompt, **generation_kwargs)[0]["generated_text"]
    outputs.append({"prompt": prompt, "response": response})
    print("\n---\n")
    print(response)

with open(OUTPUT_DIR / "generations.json", "w", encoding="utf-8") as fp:
    json.dump(outputs, fp, indent=2)


## Measurement


In [ ]:

def run_inference(recorder):
    for idx, prompt in enumerate(prompts):
        result = generator(prompt, **generation_kwargs)
        if idx == 0:
            recorder.mark_first_token()
        recorder.add_items(len(result[0]["generated_text"].split()))

metrics = measure_memory_speed(run_inference)

def fmt(value, digits=4):
    if value in (None, "", float("inf")):
        return ""
    return f"{value:.{digits}f}"

try:
    repo_commit = subprocess.check_output(["git", "rev-parse", "HEAD"], text=True).strip()
except Exception:  # noqa: BLE001
    repo_commit = ""

append_benchmark_row(
    task="instruction-generation",
    model_id=MODEL_ID,
    dataset="synthetic-prompts",
    sequence_or_image_res="128-tokens",
    batch="1",
    peak_ram_mb=fmt(metrics.get("peak_ram_mb"), 2),
    peak_vram_mb=fmt(metrics.get("peak_vram_mb"), 2),
    load_time_s=fmt(load_time, 2),
    ttfb_s=fmt(metrics.get("ttfb_s"), 3),
    tokens_per_s_or_images_per_s=fmt(metrics.get("throughput_per_s"), 3),
    precision=PRECISION,
    notebook_path="notebooks/nlp/instruct-generation-llama-3-instruct-8b_lite_cpu-first.ipynb",
    repo_commit=repo_commit,
)

with open(OUTPUT_DIR / "metrics.json", "w", encoding="utf-8") as fp:
    json.dump(metrics, fp, indent=2)
metrics


## Results Summary
        - Observations: TODO
        - Metrics captured: see `benchmarks/matrix.csv`

        ## Next Steps
        - TODOs: fill in after benchmarking

        ## Repro
        - Seed: 42 (set in measurement cell)
        - Libraries: captured via `detect_env()`
        - Notebook path: `notebooks/nlp/instruct-generation-llama-3-instruct-8b_lite_cpu-first.ipynb`
        - Latest commit: populated automatically when appending benchmarks (if git available)
